In [4]:
import cv2
import torch
from tracker import *
import numpy as np
import mediapipe as mp
import playsound
from mutagen.mp3 import MP3
import time
import miniaudio

#get pose from mediapipe
mpPose = mp.solutions.pose
pose = mpPose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)
mpDraw = mp.solutions.drawing_utils



model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

cap=cv2.VideoCapture('vid.mp4')


tracker = Tracker()
c = set()

#calculate the angle of the elbow
def calc_angle(a,b,c):
    a = np.array(a) #armpit=shoulder
    b = np.array(b) #elbow
    c = np.array(c) #wrist
    #arctan means the absolute tan = tan^-1
    radians = np.arctan2(c[1]-b[1],c[0]-b[0]) - np.arctan2(a[1]-b[1],a[0]-b[0])
    #convert from radian to angle
    angle = np.abs(radians*180/np.pi)
    #if the arm move in 360d we will subtract it from 360 to be in range 0-180
    if angle > 180.0:
        angle = 360-angle
    return angle

file='alarm.mp3'
audio = MP3(file)
length=audio.info.length

frame_check = 7
flag = 0
#read all frames from the video
while True:
    ret,frame=cap.read()
    if ret==False:
        break
    #frame=cv2.resize(frame,(1020,500))
    results = model(frame)
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    #Apply the mediapipe pose detection module for detection
    result = pose.process(imgRGB)
    #print(results.pose_landmarks)
    h , w , c = frame.shape
    # Draw landmarks
    if result.pose_landmarks:
        #mpDraw.draw_landmarks(frame,result.pose_landmarks, mpPose.POSE_CONNECTIONS)
        #get the coor of each landmarks
        landmarks = result.pose_landmarks.landmark
        
        #use the necessary pats only which is shoulder, elbow and wrist
        #for land in mpPose.PoseLandmark:
            #print(land)
        l_shoulder = [landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value].x,
                   landmarks[mpPose.PoseLandmark.LEFT_SHOULDER.value].y]
        r_shoulder = [landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].x,
                   landmarks[mpPose.PoseLandmark.RIGHT_SHOULDER.value].y]
        
        l_elbow = [landmarks[mpPose.PoseLandmark.LEFT_ELBOW.value].x,
                   landmarks[mpPose.PoseLandmark.LEFT_ELBOW.value].y]
        r_elbow = [landmarks[mpPose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mpPose.PoseLandmark.RIGHT_ELBOW.value].y]  
             
        l_wrist = [landmarks[mpPose.PoseLandmark.LEFT_WRIST.value].x,
                   landmarks[mpPose.PoseLandmark.LEFT_WRIST.value].y]
        r_wrist = [landmarks[mpPose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mpPose.PoseLandmark.RIGHT_WRIST.value].y]

        #calculate the angel
        l_ang = calc_angle(l_shoulder,l_elbow,l_wrist)
        r_ang = calc_angle(r_shoulder,r_elbow,r_wrist)
        
        #cv2.putText(frame,str(int(l_ang)),tuple(np.multiply(l_elbow,[640,480]).astype(int)),
                    #cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),1)
        #cv2.putText(frame,str(int(r_ang)),tuple(np.multiply(r_elbow,[640,480]).astype(int)),
                    #cv2.FONT_HERSHEY_PLAIN,2,(255,255,255),1)
        #1 mean y aixs
        if l_wrist[1]*h < l_elbow[1]*h < l_shoulder[1]*h and l_ang > 150:

            #if someone still raise his hand for 7 frames,w issues a warning
            flag += 1
            if flag >= frame_check:
                cv2.putText(frame,'Warnning!!! someone need help',(20,75),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),2)
                stream = miniaudio.stream_file(file)
                with miniaudio.PlaybackDevice() as device:
                    device.start(stream)
                    time.sleep(length)
            
        elif r_wrist[1]*h < r_elbow[1]*h < r_shoulder[1]*h and r_ang > 150:
    
            flag += 1
            if flag >= frame_check:
                cv2.putText(frame,'Warnning! someone need help',(20,75),cv2.FONT_HERSHEY_PLAIN,2,(0,0,255),2)
                stream = miniaudio.stream_file(file)
                with miniaudio.PlaybackDevice() as device:
                    device.start(stream)
                    time.sleep(length)
        
        
   
    points = []
    for index , row in results.pandas().xyxy[0].iterrows():
        
        x1 = int(row['xmin'])
        y1 = int(row['ymin'])
        x2 = int(row['xmax'])
        y2 = int(row['ymax'])
        n=(row['name'])
        
        if 'person' in n:
            if row['confidence'] > 0.25:
                points.append([x1,y1,x2,y2]) 
            #cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,255),2)
            #cv2.putText(frame,str(n),(x1,y1),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),2)
    boxes_id = tracker.update(points) 
    num = len(points)
    id = []
    #person_id = []
    for box_id in boxes_id:
        x , y , w , h , idd = box_id
        id.append(idd)    
            
        #cv2.rectangle(frame,(x,y),(w,h),(0,255,0),2)
        #cv2.putText(frame,'number of persons is='+str(num),(20,50),cv2.FONT_HERSHEY_PLAIN,2,(255,0,255),3)
        #cv2.putText(frame,'person ID is='+str(id[-1]),(x,y),cv2.FONT_HERSHEY_PLAIN,1,(255,0,255),2)
 
  
    cv2.imshow('FRAME',frame)
    if cv2.waitKey(1)&0xFF==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

Using cache found in C:\Users\mosta/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-9-25 Python-3.8.10 torch-2.0.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
